# 1. Thông tin nhóm

### *Danh sách thành viên:*

<center>
    
|   MSSV   |      Họ tên      | Tài khoản Github |
|:--------:|:----------------:|:----------------:|
| 19120387 |    Lê Sỹ Thuần   |    SyThuan1803   |
| 19120119 | Nguyễn Minh Phúc |   youngtobesick  |
    
</center>

### *Kế hoạch làm việc:* 
[Xem tại đây](https://docs.google.com/spreadsheets/d/1DCq30ORIH6m-1EYMnbUmZ8fBoxPiiCgRgLDpJaRKRVE/edit?usp=sharing)

---

# 2. Thu thập dữ liệu

#### Dữ liệu của bạn là về chủ đề gì và bạn lấy từ nguồn nào?

Dữ liệu là bảng đánh giá độ hài lòng của các khách về các khách sạn Châu Âu
</br> Dữ liệu được lấy từ nguồn được public trên [Kaggle](https://www.kaggle.com/ishansingh88/europe-hotel-satisfaction-score)

**Phần mô tả của dữ liệu**

Phần mô tả dữ liệu không có gì đặc biệt, chỉ là các dòng mô tả giá trị của các cột có trong dataset.  
Dưới đây là phần mô tả được lấy từ trang publish dataset:  

Columns
Age - 7 to 85

purposeoftravel - aviation, academic, personal, business, tourism.

Type of Travel - Group travel, Personal Travel.

Type Of Booking - Group bookings, Individual/Couple.

Hotel wifi service - Ratings out of 5.

Departure/Arrival convenience - Ratings out of 5.

Ease of Online booking - Ratings out of 5.

Hotel location - Ratings out of 5.

Food and drink - Ratings out of 5.

Stay comfort - Ratings out of 5.

Common Room entertainment - Ratings out of 5.

Checkin/Checkout service - Ratings out of 5.

Other service - Ratings out of 5.

Cleanliness - Ratings out of 5.

satisfaction - satisfied, neutral or dissatisfied.

**License của dữ liệu**

License của dữ liệu là Unknown ([nguồn](https://www.kaggle.com/ishansingh88/europe-hotel-satisfaction-score/metadata)).  
Có vẻ như người publish dữ liệu này lên Kaggle không có yêu cầu gì đối với việc sử dụng dữ liệu, hoặc cũng có thể là quên cập nhật chi tiết License. Nhưng có lẽ việc sử dụng dữ liệu với mục đích học tập thì cũng không đến nỗi nào 😓.

#### Người ta thu thập dữ liệu từ nguồn nào?

Em đã tìm hết tất cả tab ở link bên trên nhưng vẫn không thấy chỗ nào có ghi cách để thu thập bộ dữ liệu này.

Em đoán là có thể lúc khách trả phòng, họ được nhờ để điền mẫu khảo sát về các dịch vụ của khách sạn để thu thập dữ liệu. Vì title chỉ nói chung chung là Europe Hotel mà không để chi tiết là một hay nhiều hotel nào nên cũng có thể là nhóm thu thập dữ liệu tiến hành khảo sát trên cộng đồng nào đó để có thể đưa ra một bộ dữ liệu đồng nhất về các thuộc tính như thế này.

---

# 3. Import các thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.tree import export_text

---

# 4. Khám phá & tiền xử lý dữ liệu

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [2]:
# ehs là viết tắt của europe hotel satisfaction
ehs_df = pd.read_csv('./data/Europe Hotel Booking Satisfaction Score.csv')
ehs_df.head()

,id,Gender,Age,purpose_of_travel,Type of Travel,Type Of Booking,Hotel wifi service,Departure/Arrival convenience,Ease of Online booking,Hotel location,Food and drink,Stay comfort,Common Room entertainment,Checkin/Checkout service,Other service,Cleanliness,satisfaction
0,70172,Male,13,aviation,Personal Travel,Not defined,3,4,3,1,5,5,5,4,5,5,neutral or dissatisfied
1,5047,Male,25,tourism,Group Travel,Group bookings,3,2,3,3,1,1,1,1,4,1,neutral or dissatisfied
2,110028,Female,26,tourism,Group Travel,Group bookings,2,2,2,2,5,5,5,4,4,5,satisfied
3,24026,Female,25,tourism,Group Travel,Group bookings,2,5,5,5,2,2,2,1,4,2,neutral or dissatisfied
4,119299,Male,61,aviation,Group Travel,Group bookings,3,3,3,3,4,5,3,3,3,3,satisfied


### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột?

In [3]:
num_rows,num_cols = ehs_df.shape
print(f'Number of rows: {num_rows}')
print(f'Number of cols: {num_cols}')

Number of rows: 103904
Number of cols: 17


Vậy là dữ liệu được thu thập hơn 100 nghìn dòng (có thể tồn tại các dòng bị trùng nhau) với khoảng 15 thuộc tính được khảo sát.

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Theo mô tả của file dữ liệu và theo quan sát sơ bộ thì mỗi dòng đại diện cho một khảo sát của một người cụ thể với các dịch vụ của khách sạn, và nhìn chung thì em hy vọng là không có dòng nào có ý nghĩa khác với những dòng còn lại (nếu có thì trong quá trình khám phá và phân tích, ta có thể lược bỏ các dòng này sau).

### Dữ liệu có các dòng bị lặp không?

In [4]:
print(f'Duplicated rows in dataframe: {ehs_df.duplicated().sum()}')

Duplicated rows in dataframe: 0


Okay, theo như kết quả của của dòng code phía trên thì dataframe của ta không tồn tại các dòng bị trùng nhau, ta có thể tiếp tục khám phá mà không cần ở lại xử lý vấn đề bị lặp dòng này.

### Mỗi cột có ý nghĩa gì?

Tuy phần thông tin mô tả dữ liệu đã đề cập đến tên các cột và giá trị của các cột này, nhưng nó không đề cập đến ý nghĩa của tên cột nên em nghĩ là sẽ giải đáp một cách khá tương đối về vấn đề này và hy vọng là không có sự hiểu nhầm ở đây 😉.

Sau đây là ý nghĩa của các cột và các giá trị (được trích từ thông tin mô tả dữ liệu, các dòng không để giá trị ở dưới đây mặc định là điểm ratings từ 0-5):

|Thuộc tính|    Ý nghĩa   |
|:--------:|:-----:|
|**id**|ID của survey||
|**Gender**|Giới tính người thực hiện survey|
|**Age**|Tuổi của đối tượng khảo sát (từ 7-85)| 
|**purpose_of_travel**|Mục đích của chuyến đi trong lúc sử dụng khách sạn (gồm aviation, academic, personal, business, tourism)|
|**type_of_travel**|Loại chuyến đi (gồm Group travel, Personal Travel)|
|**Type of Booking**|Loại đặt phòng (gồm Group bookings, Individual/Couple)|
|**Hotel wifi service**|Wifi của khách sạn|
|**Departure/Arrival  convenience**|Tiện nghi khi đến|
|**Ease of Online booking**|Thoải mái booking online|
|**Hotel location**|Địa điểm hotel|
|**Food and drink**|Thức ăn và đồ uống|
|**Stay comfort**|Chỗ ở thoải mái|
|**Common Room entertainment**|Phòng giải trí chung|
|**Checkin/Checkout service**|Dịch vụ check in/ check out|
|**Other service**|Dịch vụ khác|
|**Cleanliness**|Sạch sẽ|
|**satisfaction**|Độ hài lòng của khách (gồm satisfied, neutral or dissatisfied)|

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

Trước khi xem kiểu dữ liệu của các cột, ta cần format tên cột theo một chuẩn nhất định để dễ thao tác hơn. Chuẩn được đặt ra ở đây đó chính là viết thường các ký tự tên cột và thay thế khoảng trắng bằng dấu gạch dưới (underscore)

In [5]:
def convert_name(old_name):
    new_name = old_name.lower()
    new_name = new_name.replace('  ', '_')
    new_name = new_name.replace(' ', '_')
    new_name = new_name.replace('/', '_')
    return new_name

ehs_df.rename(columns=convert_name, inplace=True)
ehs_df.columns

Index(['id', 'gender', 'age', 'purpose_of_travel', 'type_of_travel',
       'type_of_booking', 'hotel_wifi_service',
       'departure_arrival_convenience', 'ease_of_online_booking',
       'hotel_location', 'food_and_drink', 'stay_comfort',
       'common_room_entertainment', 'checkin_checkout_service',
       'other_service', 'cleanliness', 'satisfaction'],
      dtype='object')

In [6]:
col_dtypes = ehs_df.dtypes
col_dtypes

id                                int64
gender                           object
age                               int64
purpose_of_travel                object
type_of_travel                   object
type_of_booking                  object
hotel_wifi_service                int64
departure_arrival_convenience     int64
ease_of_online_booking            int64
hotel_location                    int64
food_and_drink                    int64
stay_comfort                      int64
common_room_entertainment         int64
checkin_checkout_service          int64
other_service                     int64
cleanliness                       int64
satisfaction                     object
dtype: object

Hiện tại thì `ehs_df` chỉ có 5 cột có `dtype` là `object`: "gender", "purpose_of_travel", "type_of_travel", "type_of_booking", "satisfaction". Chúng ta sẽ thử xem các cột này đang giữ các kiểu dữ liệu nào.

In [7]:
def open_object_dtype(s):
    return set(s.apply(type))

obj_cols_name = ["gender", "purpose_of_travel",
                 "type_of_travel", "type_of_booking", "satisfaction"]

for col_name in obj_cols_name:
    print(f"{col_name}: {open_object_dtype(ehs_df[col_name])}")

gender: {<class 'str'>}
purpose_of_travel: {<class 'str'>}
type_of_travel: {<class 'str'>}
type_of_booking: {<class 'str'>}
satisfaction: {<class 'str'>}


**Vấn đề cần tiền xử lý:**
- Cột Type Of Booking có giá trị là Not defined tương tự với việc đánh dấu giá trị thiếu, nên cần phải điền vào những ô có giá trị này.

## Tiền xử lý

**Thay thế những ô có giá trị Not defined trong cột Type of booking**

*Phương pháp:*
- Thay thế giá trị Not defined bằng 2 giá trị còn lại trong cột Type of booking bằng phương pháp max similarity (độ tương tự lớn nhất)

Chuẩn hóa cột categorical ề

In [8]:
# Chuyển hết giá trị Not defined thành NaN
ehs_df.loc[ehs_df['type_of_booking'] == 'Not defined', 'type_of_booking'] = np.nan

In [9]:
# Chuẩn hóa các cột Categorical thành cột có dạng số
#Tạo một bản sao data frame để chuẩn hóa
ehs_df_normalize=ehs_df.copy()
ehs_df_normalize['gender'] = ehs_df_normalize['gender'].replace(['Male'],1)
ehs_df_normalize['gender'] =ehs_df_normalize['gender'].replace(['Female'],2)

ehs_df_normalize['type_of_travel'] = ehs_df_normalize['type_of_travel'].replace(['Personal Travel'],1)
ehs_df_normalize['type_of_travel'] = ehs_df_normalize['type_of_travel'].replace(['Group Travel'],2)

ehs_df_normalize['satisfaction'] = ehs_df_normalize['satisfaction'].replace(['neutral or dissatisfied'],0)
ehs_df_normalize['satisfaction'] = ehs_df_normalize['satisfaction'].replace(['satisfied'],1)
ehs_df_normalize['type_of_booking'] = ehs_df_normalize['type_of_booking'].replace(['Individual/Couple'],1)
ehs_df_normalize['type_of_booking'] = ehs_df_normalize['type_of_booking'].replace(['Group bookings'],2)

ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['tourism'],5)
ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['academic'],4)
ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['business'],3)
ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['aviation'],2)
ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['personal'],1)

Chọn ra tất cả các dòng có giá trị NaN

In [10]:
sample_df_isnan=ehs_df_normalize[ehs_df_normalize['type_of_booking'].isna()]
sample_df_isnan

,id,gender,age,purpose_of_travel,type_of_travel,type_of_booking,hotel_wifi_service,departure_arrival_convenience,ease_of_online_booking,hotel_location,food_and_drink,stay_comfort,common_room_entertainment,checkin_checkout_service,other_service,cleanliness,satisfaction
0,70172,1,13,2,1,NaN,3,4,3,1,5,5,5,4,5,5,0
11,51412,2,12,5,1,NaN,2,4,2,2,1,1,1,5,5,1,0
27,14925,2,49,3,2,NaN,4,4,4,4,2,1,4,2,4,2,1
37,48014,1,40,1,1,NaN,4,3,4,2,2,2,2,3,4,2,0
69,9261,2,38,1,2,NaN,4,4,4,4,4,4,4,1,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103811,12866,1,27,1,2,NaN,4,4,4,4,4,4,4,3,4,4,0
103867,81857,1,30,1,2,NaN,3,2,2,2,3,3,3,2,3,3,0
103868,37051,2,57,1,1,NaN,1,5,1,5,4,3,5,1,5,1,0
103877,14662,1,27,5,2,NaN,5,4,4,4,5,5,5,2,3,5,1


Có tổng cộng 7494 dòng cần điền.  
Do số lượng dữ liệu quá nhiều dẫn đến tốn nhiều trong lúc thực hiện nên nhóm sẽ lấy mẫu bao gồm 1000 dòng không chứa giá trị NaN để suy ra giá trị NaN.  
Lấy random mẫu số lượng 1000 dòng không chứa giá trị NaN và nối vào dataframe chứa giá trị NaN ở trên.

In [11]:
sample_df=ehs_df_normalize[~ehs_df_normalize['type_of_booking'].isna()]
sample_df=sample_df.sample(frac=0.01, random_state=30).reset_index(drop=True)
sample_df1=pd.concat((sample_df_isnan,sample_df),axis=0).reset_index(drop=True)
sample_df1

,id,gender,age,purpose_of_travel,type_of_travel,type_of_booking,hotel_wifi_service,departure_arrival_convenience,ease_of_online_booking,hotel_location,food_and_drink,stay_comfort,common_room_entertainment,checkin_checkout_service,other_service,cleanliness,satisfaction
0,70172,1,13,2,1,NaN,3,4,3,1,5,5,5,4,5,5,0
1,51412,2,12,5,1,NaN,2,4,2,2,1,1,1,5,5,1,0
2,14925,2,49,3,2,NaN,4,4,4,4,2,1,4,2,4,2,1
3,48014,1,40,1,1,NaN,4,3,4,2,2,2,2,3,4,2,0
4,9261,2,38,1,2,NaN,4,4,4,4,4,4,4,1,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8453,61001,2,45,5,2,1.0,2,1,2,3,5,5,5,1,2,5,0
8454,112706,1,39,5,2,1.0,4,4,4,3,1,1,1,1,4,1,0
8455,2663,2,53,3,2,2.0,5,5,5,5,3,5,4,5,4,5,1
8456,46666,1,30,4,2,2.0,4,1,1,1,2,4,2,1,3,2,0


Tiến hành tính toán độ tương tự lớn nhất và dự đoán giá trị bị thiếu

In [12]:
# Đưa dữ liệu của dataframe về numpy array để bắt đầu xử lý tính độ tương tự lớn nhất
ratings=sample_df1.drop('id',axis=1)
ratings['type_of_booking']=pd.to_numeric(ratings['type_of_booking'])
ratings=ratings.astype('float64')
ratings=np.array(ratings.iloc[:,:])
ratings

array([[ 1., 13.,  2., ...,  5.,  5.,  0.],
       [ 2., 12.,  5., ...,  5.,  1.,  0.],
       [ 2., 49.,  3., ...,  4.,  2.,  1.],
       ...,
       [ 2., 53.,  3., ...,  4.,  5.,  1.],
       [ 1., 30.,  4., ...,  3.,  2.,  0.],
       [ 2., 23.,  4., ...,  4.,  5.,  1.]])

In [ ]:
# Tiến hành tính giá trị cần điền bằng độ tương tự lớn nhất
batch_size = 32
filled_ratings = np.empty_like(ratings)
start = 0
end = batch_size
new_ratings=np.tile(ratings,(batch_size,1))
while(1):
    if (start>=len(ratings)):
        break
    r=ratings[start:end,:]
    if batch_size != 32:
        new_ratings=np.tile(ratings,(batch_size,1))
    r= np.repeat(r, repeats=len(ratings), axis=0)
    a=np.abs(new_ratings[:,:] -r)
    #Tao similarities
    similarities=np.zeros((batch_size,len(ratings)))
    startRow=0
    for ind in range(batch_size):
        similarities[ind]=(1/(np.nanmean(a[startRow:startRow+len(ratings)],axis=1)+0.001))
        startRow+=len(ratings)
    similarities=np.nan_to_num(similarities,nan=0) 
    weights=np.zeros((batch_size,len(ratings),ratings.shape[1]))
    for ind in range(batch_size):
        normalize_w=~np.isnan(ratings)*similarities[ind].reshape(-1,1)
        weights[ind]=normalize_w/normalize_w.sum(axis=0)
    filled_ratingsBatch=np.nansum(ratings*weights,axis=1)
    filled_ratings[start:end]=filled_ratingsBatch
    start=end
    end=start+batch_size
    if(end>len(ratings)):
        end=len(ratings)
        batch_size=end-start

Điền giá trị thiếu vào mẫu đã trích ra tất cả dòng thiếu NaN, sau đó nối vào dataframe không có giá trị NaN.

In [ ]:
filled_nanvals = filled_ratings[np.isnan(ratings)].round(0)
#Điền giá trị bị thiếu
sample_df_isnan=ehs_df[ehs_df['type_of_booking'].isna()]
sample_df_isnan.loc[:,'type_of_booking']=filled_nanvals
#Chuyển về lại dạng categorical
sample_df_isnan.loc[sample_df_isnan['type_of_booking'] == 1, 'type_of_booking'] ='Individual/Couple' 
sample_df_isnan.loc[sample_df_isnan['type_of_booking'] == 2, 'type_of_booking'] ='Group bookings' 
#Tiến hành nối dataframe không chứa giá trị NaN và dataframe với giá trị NaN đã xử lí
ehs_df=ehs_df[~ehs_df['type_of_booking'].isna()]
ehs_df=pd.concat((sample_df_isnan,ehs_df),ignore_index=True,axis=0)
ehs_df=ehs_df.sample(frac=1, random_state=42).reset_index(drop=True)
ehs_df

In [ ]:
# Kiểm tra còn dòng NaN không
ehs_df[ehs_df['type_of_booking'].isna()]

Dữ liệu đã được điền đầy đủ.

---

## Khám phá dữ liệu (tiếp tục)

### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

Sau khi thực hiện bước tiền xử lý phía trên thì ta xếp 11 cột sau vào nhóm numeric: "age", "hotel_wifi_service", "departure_arrival_convenience", "ease_of_online_booking", "hotel_location", "food_and_drink", "stay_comfort", "common_room_entertainment", "checkin_checkout_service", "other_service", "cleanliness".

Các cột còn lại mặc dù có kiểu số nhưng thật ra có dạng categorical (vì là mã được đánh dưới dạng số). Như vậy, chỉ có 11 cột numeric và các cột ngoại trừ "age" đều là thang điểm từ 0-5.

Với mỗi cột numeric ta sẽ tính tỉ lệ % giá trị thiếu (từ 0 đến 100), min, max và lưu kết quả vào dataframe nume_col_profiles_df. Dataframe này có 4 dòng là "missing_ratio", "min", "mean", "max"; và có 11 cột có tên được nêu ở trên.

In [ ]:
def missing_ratio(s):
    return s.isna().mean() * 100

cate_col_name = ["id", "gender", "purpose_of_travel", "type_of_travel", "type_of_booking", "satisfaction"]
nume_col_idx = ehs_df.columns.drop(cate_col_name)
nume_col_df = ehs_df[nume_col_idx]
nume_col_profiles_df = nume_col_df.agg([missing_ratio, pd.Series.min,pd.Series.mean, pd.Series.max])

nume_col_profiles_df

**Nhận xét các dữ liệu số:**
- Độ tuổi trung bình của người thực hiện khảo sát là 39 tuổi, nhỏ nhất là 7 tuổi, lớn nhất là 85 tuổi 
- Các chỉ số đánh giá mức độ hài lòng từ 0 đến 5
- Các giá trị trung bình của chỉ số đánh giá nằm từ 2 đến 3, nhìn chung cũng khá cân bằng

Ta sẽ xem rõ hơn về sự phân bố của các giá trị trong mỗi cột có kiểu là numeric bằng hàm plot_hist_cols với hai tham số truyền vào là dataframe và số cột.

In [ ]:
def plot_hist_cols(df, num_graph_per_row):
    n_rows, n_cols = df.shape
    col_names = list(df.columns)
    n_rows_graph = (n_cols + num_graph_per_row - 1) / num_graph_per_row
    
    plt.figure(
        figsize = (6 * num_graph_per_row,  6 * n_rows_graph),
        dpi = 80,
    )
    
    for i in range(n_cols):
        plt.subplot(int(n_rows_graph), num_graph_per_row, i + 1)
        col_i = df.iloc[:, i]
        
        if (not np.issubdtype(type(col_i.iloc[0]), np.number)):
            val_counts = col_i.value_counts()
            plt.xticks(rotation = 90)
            val_counts.plot.bar()
        else:
            nunique_col_i = col_i.nunique()
            bins = nunique_col_i if (nunique_col_i < 10) else 10
            col_i.hist(bins=bins)
        
        plt.ylabel('counts')
        plt.title(f'{col_names[i]}')
        
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.show()

In [ ]:
plot_hist_cols(nume_col_df, 4)

**Nhận xét:**
- Gender: có 2 loại nam, nữ
- Purpose_of_travel: có 5 giá trị aviation(hàng không), tourism(theo tour), business(công việc), academic(trường học), personal(cá nhân)
- Type of travel: Personal Travel(cá nhân), Group Travel(theo nhóm)
- Type of booking: Not defined(không xác định), Group bookings(theo nhóm), Individual/Couple(cá nhân/ cặp đôi)
- Satisfaction: neutral or dissatisfied(bình thường hoặc không hài lòng), satisfied(hài lòng)

### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

`ehs_df` có 5 cột categorical là "id", "gender", "purpose_of_travel", "type_of_travel", "type_of_booking".

Với mỗi cột categorical, ta tính tỉ lệ % giá trị thiếu (từ 0 đến 100), số lượng giá trị khác nhau (không xét giá trị thiếu), list/array các giá trị khác nhau (không xét giá trị thiếu), và lưu kết quả vào dataframe cate_col_profiles_df. Dataframe này có 3 dòng là "missing_ratio", "num_diff_vals", "diff_vals"; và có 5 cột là "id", "gender", "purpose_of_travel", "type_of_travel", "type_of_booking".

In [ ]:
cate_col_df = ehs_df[cate_col_name]

cate_col_profiles_df = cate_col_df.agg([missing_ratio, pd.Series.nunique, pd.Series.unique])

cate_col_profiles_df.rename(
    index={'nunique': 'num_diff_vals', 'unique': "diff_vals"},
    inplace=True
)

cate_col_profiles_df

Ta sẽ xem rõ hơn về sự phân bố của các giá trị trong mỗi cột có kiểu là categorical

In [ ]:
plot_hist_cols(cate_col_df, 3)

---

# 5. Đưa ra các câu hỏi cần trả lời

#### Câu 1: Mục đích chuyến đi có sự đánh giá độ hài lòng khác nhau như thế nào?
Trả lời được câu hỏi này sẽ cho ta biết được các tiêu chí trong mục đích chuyến đi để chuẩn bị tốt hơn cho các hợp đồng của công ty với khách sạn sau này.

#### Câu 2: Các yếu tố ảnh hưởng đến sự hài lòng giữa nam và nữ khác nhau như thế nào?
Trả lời được câu hỏi giúp khách sạn có những định hướng cụ thể để cải thiện các tiêu chí đối với mỗi giới tính, qua đó có thể cải thiện độ hài lòng của khách hàng.

#### Câu 3: Trong tất cả các yếu tố được chấm điểm, những yếu tố nào quan trọng nhất để một khách sạn sẽ được đánh giá hài lòng? 
Trả lời được câu hỏi này để nếu ở Việt Nam muốn mở một khách sạn thì sẽ tham khảo những yếu tố này để cải thiện, nâng cấp.

#### Câu 4: Độ tuổi của khách hàng đối với từng mục đích chuyến đi ảnh hưởng như thế nào đến sự hài lòng?
Trả lời được câu hỏi này sẽ giúp khách sạn có những phương án phù hợp tùy theo độ tuổi và mục đích chuyến đi để phát triển, đáp ứng nhu cầu của khách hàng.

---

# 6.Tiền xử lý, phân tích dữ liệu để trả lời cho từng câu hỏi

## 6.1 Câu hỏi 1:  Mục đích chuyến đi có sự đánh giá độ hài lòng khác nhau như thế nào?


### Tiền xử lý

Chuyển cột satisfaction về 0 1 để tính toán nhanh hơn, `neutral or dissatisfied` là 0, `satisfaction` là 1

In [ ]:
ehs_df_normalize=ehs_df.copy()
ehs_df_normalize['satisfaction'] = ehs_df_normalize['satisfaction'].replace(['neutral or dissatisfied'],0)
ehs_df_normalize['satisfaction'] = ehs_df_normalize['satisfaction'].replace(['satisfied'],1)

### Phân tích dữ liệu để trả lời cho các câu hỏi 

<b>Đầu tiên, xem tỉ lệ đánh giá độ hài lòng của từng mục đích chuyến đi

In [ ]:

average_satisfaction=ehs_df_normalize.groupby('purpose_of_travel')['satisfaction'].mean()*100
average_satisfaction

Tỉ lệ hài lòng trung bình của các mục đích chuyến đi:
</br>  aviation: 42,67%
</br>  academic: 43,16%
</br>  business: 43,58%
</br>  personal: 43,62%
</br>  tourism: 43,52%

In [ ]:
colors = ['#1b9e77', '#a9f971', '#fdaa48','#6890F0','#A890F0']
plt.barh(average_satisfaction.index,average_satisfaction.values,color=colors);
plt.title("Average satisfaction of purpose travel")
plt.ylabel("purpose_of_travel")
plt.xlabel("%")
plt.show()

**Nhận xét:**
- Các mục đích chuyến đi có tỉ lệ đánh giá độ hài lòng về khách sạn Châu Âu tương đương nhau, đều có đánh giá hài lòng ít hơn 50%, trong đó aviation có tỉ lệ đánh giá thấp nhất.
- Aviation là chuyến đi của ngành hàng không, bao gồm phi công, tiếp viên. Họ có thể đã đi nhiều nơi nên việc đánh giá trở nên khắt khe hơn.

<b>Tìm hiểu sâu hơn về các điểm số đánh giá tiêu chí của mục đích chuyến đi có sự hài lòng về khách sạn

Xem điểm số trung bình của môi tiêu chí đánh giá ứng với từng mục đích chuyến đi

In [ ]:
ehs_df_normalize=ehs_df_normalize[ehs_df_normalize['satisfaction']==1]

score_cols=['hotel_wifi_service',
       'departure_arrival_convenience', 'ease_of_online_booking',
       'hotel_location', 'food_and_drink', 'stay_comfort',
       'common_room_entertainment', 'checkin_checkout_service',
       'other_service', 'cleanliness']
avg_score=ehs_df_normalize.groupby('purpose_of_travel')[score_cols].mean()
avg_score

Vẽ biểu đồ thể hiện độ lệch giữa trung bình điểm số đánh giá của từng cặp mục đích chuyến đi

In [ ]:
purpose_travel_cols=['academic','aviation','business','personal','tourism']

len_purpose=len(purpose_travel_cols)
for i in range(len_purpose):
    for j in range(i+1,len_purpose):
        plt.barh(avg_score.columns,avg_score.iloc[i]-avg_score.iloc[j],color=colors)
        plt.title(f'{purpose_travel_cols[i]} with {purpose_travel_cols[j]}')
        plt.show()

**Nhận xét:**
- Từ các biểu đồ này ta sẽ nhìn thấy được sự khác nhau rõ hơn trong các tiêu chí đánh giá của từng hạng mục về độ hài lòng.
- Ví dụ như personal và tourism:
</br>- Tourism sẽ đánh giá stay_comfort, food_and_drink, hotel_location, departure_arrival_convenience khắt khe hơn, đồng nghĩa với việc sẽ quan tâm những tiêu chí này hơn.
</br>- Ngược lại những tiêu chí như cleanlliness, checkin checkout service... sẽ được bên personal chấm điểm cao hơn.
> <I> Từ đó, ta sẽ có thêm một tiêu chí phụ nữa để đưa ra các quyết định thay đổi, nâng cấp khách sạn ứng với từng hợp đồng của các bên đối tác về trường học, tour, hàng không... Nắm bắt được điểm yếu của mình sẽ kịp thời sửa chữa và thay đổi nhằm nâng cao hơn chất lượng của khách sạn

---

## 6.2 Câu hỏi 2: Các yếu tố ảnh hưởng đến sự hài lòng giữa nam và nữ khác nhau như thế nào?

### Tiền xử lý

- Đổi các giá trị ở cột satisfaction thành 1 cho "satisfied" và 0 cho "neutral or dissatisfied" để tiện cho việc xử lý, khám phá và phân tích dữ liệu

In [ ]:
ehs_df['satisfaction'].replace(
    to_replace=['neutral or dissatisfied', 'satisfied'],
    value=[0, 1],
    inplace=True
)

### Phân tích dữ liệu để trả lời cho các câu hỏi 

**Xem số lượng khách hàng nam và nữ thỏa mãn các dịch vụ của khách sạn**

In [ ]:
sns.countplot(x='gender',hue='satisfaction',data=ehs_df)

**Phần trăm khách hàng hài lòng dựa theo giới tính**

In [ ]:
plt.figure(figsize=(12, 6))
genders = ['Male', 'Female']

for gender in genders:
    plt.subplot(1, 2, genders.index(gender)+1)
    plt.pie(
        ehs_df.loc[ehs_df.gender==gender, 'satisfaction'].value_counts(),
        labels=['neutral or dissatisfied','satisfied'],
        explode=[0, 0.05],
        shadow=True,
        autopct='%1.2f%%'
    )
    plt.title(f'Satisfaction ({gender})', fontsize=15)

plt.show()

In [ ]:
group_data = ehs_df.groupby(['gender']).mean().T[2:-1]
group_data.plot.barh(
    figsize=(8,8),
    title = 'Điểm trung bình các yếu tố theo giới tính'
)
plt.show()

***Nhận xét:*** 
- Có thể thấy rằng phân bố theo giới tính của khách hàng hài lòng và không hài lòng là khá giống nhau. Đối với cả khách nam và nữ, số lượng các khách hàng không hài lòng cao hơn so với số lượng khách hàng hài lòng
- Tỷ lệ hài lòng của khách hàng nam là 44%, của khách hàng nữ 43%
- Trung bình ratings của các yếu tố của nam và nữ khá đồng đều và gần như không có sự khác biệt

In [ ]:
male_ratings_df = ehs_df[ehs_df['gender'] == 'Male']
s1 = male_ratings_df.corr()['satisfaction'][2:-1].rename('Male')

female_ratings_df = ehs_df[ehs_df['gender'] == 'Female']
s2 = female_ratings_df.corr()['satisfaction'][2:-1].rename('Female')

gender_corr = pd.concat([s1, s2], axis=1)
gender_corr.plot.barh(
    figsize=(12,8),
    title = 'Tương quan giữa độ hài lòng và các yếu tố theo giới tính'
)

***Trả lời câu hỏi***
- Các dịch vụ khác ("other service") có mối tương quan khá cao đối với các khách hàng là nữ giới
- Các khách hàng nam có tương quan cao hơn khá đáng kể so với nữ ở các tiêu chí như "cleanliness", "stay_comfort", "food_and_drink"
- Ngoài ra độ tương quan của các tiêu chí "common_room_entertainment", "hotel_wifi_service" không có sự khác biệt rõ rệt, tuy nhiên đây cũng là các tiêu chí mà khách sạn cần quan tâm vì có độ tương quan lớn để giúp cải thiện sự hài lòng của khách hàng 

---

## 6.3 Câu hỏi 3: Trong tất cả các yếu tố được chấm điểm, những yếu tố nào quan trọng nhất để một khách sạn sẽ được đánh giá hài lòng? 

**Các bước thực hiện:**

Trong câu hỏi này, em sẽ sử dụng Machine Learning để giải quyết vấn đề. Dùng cây quyết định với phương pháp tính entropy nhỏ nhất sẽ chọn ra được 5 yếu tố chính quyết định sự hài lòng hoặc không hài lòng về khách sạn. 
- Bước 1:  Chuẩn hóa các cột categorical về numerical
- Bước 2: Chia ra các x_train, y_train và fit vào cây quyết định sử dụng thư viện sklearn- decision tree
- Bước 3: Chọn ra 5 thuộc tính đứng đầu quyết định sự hài lòng hoặc không hài lòng

### Tiền xử lý

Chuẩn hóa các cột categorical về numerical

In [ ]:
ehs_df_normalize=ehs_df.copy()
ehs_df_normalize['gender'] = ehs_df_normalize['gender'].replace(['Male'],1)
ehs_df_normalize['gender'] =ehs_df_normalize['gender'].replace(['Female'],2)

ehs_df_normalize['type_of_travel'] = ehs_df_normalize['type_of_travel'].replace(['Personal Travel'],1)
ehs_df_normalize['type_of_travel'] = ehs_df_normalize['type_of_travel'].replace(['Group Travel'],2)

ehs_df_normalize['satisfaction'] = ehs_df_normalize['satisfaction'].replace(['neutral or dissatisfied'],0)
ehs_df_normalize['satisfaction'] = ehs_df_normalize['satisfaction'].replace(['satisfied'],1)
ehs_df_normalize['type_of_booking'] = ehs_df_normalize['type_of_booking'].replace(['Individual/Couple'],1)
ehs_df_normalize['type_of_booking'] = ehs_df_normalize['type_of_booking'].replace(['Group bookings'],2)

ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['tourism'],5)
ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['academic'],4)
ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['business'],3)
ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['aviation'],2)
ehs_df_normalize['purpose_of_travel'] = ehs_df_normalize['purpose_of_travel'].replace(['personal'],1)

### Phân tích dữ liệu để trả lời cho các câu hỏi 

Xây dựng cây quyết định

In [ ]:
# Lấy khoảng 80% mẫu cho tập train, 20% cho tập test
#Không lấy cột id
ehs_df_normalize.drop('id',axis=1,inplace=True)
x=ehs_df_normalize.iloc[:,0:-1]
y=ehs_df_normalize.iloc[:,-1]
#Tách bộ train, test
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
estimator = DecisionTreeClassifier(criterion='entropy',max_depth=8,random_state=42)
# Tạo 1 selector chọn ra thuộc tính tốt nhất
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(x_train,y_train)
topatt=[i for i in range(len(selector.ranking_)) if selector.ranking_[i]==1]
result = [list(ehs_df_normalize.columns)[i] for i in topatt]
print('Top 5 attribute affect the customer satisfaction: ')
print(result)
#Tính độ chính xác
modelDT=DecisionTreeClassifier(criterion='entropy',max_depth=8,random_state=42)
modelDT.fit(x_train,y_train)
modelDT.predict(x_test)
acc=modelDT.score(x_test,y_test)*100
print(f'Accuracy: {round(acc,2)}%')



Vẽ cây quyết định

In [ ]:
r = export_text(modelDT, feature_names=list(ehs_df_normalize.columns)[0:-1])
print(r)

**Nhận xét:**
- Sau khi sử dụng phương pháp máy học là cây quyết định thì top 5 những thuộc tính quan trọng nhất ảnh hưởng đến độ hài lòng khách hàng là:
1. type_of_travel: loại hình du lịch
2. type_of_booking: loại booking
3. hotel_wifi_service: dịch vụ wifi
4. hotel_location: vị trí khách sạn
5. common_room_entertainment: phòng giải trí chung
-  Khai tác tốt vấn đề bằng việc duy trì ổn định, nâng cấp, cài thiện các yếu tố kể trên sẽ giúp khách sạn nhận được nhiều ý kiến đánh giá tích cực hơn, thu hút khách hàng mới và giữ lại tệp khách hàng cũ
- Sử dụng mô hình học máy này còn giúp phân lớp tệp khách hàng mới, biết được yêu cầu cho từng tiêu chí, từ đó nâng cao chất lượng phục vụ


---

## 6.4 Câu hỏi 4: Độ tuổi của khách hàng đối với từng mục đích chuyến đi ảnh hưởng như thế nào đến sự hài lòng?

### Tiền xử lý

### Phân tích dữ liệu để trả lời cho các câu hỏi 

- Xem các số lượng khách hàng nam và nữ thỏa mãn các dịch vụ của khách sạn

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot(
        x='age',
        data=ehs_df,
        aspect=3.0,
        kind='count',
        hue='satisfaction',
        order=range(5, 86)
    )
    g.set_ylabels('Age vs Satisfaction')

***Nhận xét:*** Độ tuổi: Từ 7 đến 38 tuổi và từ 61 đến 79 tuổi, tỷ lệ khách hàng không hài lòng cao hơn so với khách hàng hài lòng. Ngược lại, ở độ tuổi 39-60, tỷ lệ khách hàng hài lòng cao hơn so với khách hàng không hài lòng. Căn cứ vào điều này ta sẽ chia thành 4 groups: [7,20), [20,40), [40,61), [61, 85) để tiếp tục phân tích.

**Thêm cột Age Group vào dataframe**

In [ ]:
ehs_df['age_group'] = pd.cut(ehs_df['age'], [7, 20, 40, 61, 86],
    right=False
)

In [ ]:
age_grp_table = ehs_df.pivot_table(
    index=['age_group','purpose_of_travel'],
    columns=['satisfaction'],
    aggfunc={'satisfaction':['count']},
)

len_ehs_df=len(ehs_df)

for i in range(5, 21, 5):
    ax = age_grp_table.loc[age_grp_table.index[i-5:i]].T.plot(kind="bar",figsize=(8,6),rot=0)
    plt.title(f'{age_grp_table.index[i-5:i][0][0]}')
    
    # thêm percentage trên từng cột
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/len_ehs_df)
        x, y = (p.get_x() + p.get_width()*1.1), (p.get_height()+20)
        ax.annotate(percentage, (x, y), ha='right')

***Trả lời câu hỏi***
- Trên tất cả các nhóm tuổi, số lượng khách hàng hài lòng/ không hài lòng đều theo thứ tự giảm dần: "tourism", "academic", "business", "aviation", "personal". Có thể một phần là do sự chênh lệch về số lượng mẫu của các loại mục đích này
- Khách sạn cần ưu tiên những giải pháp thích hợp và hiệu quả để có thể làm hài lòng các khách hàng "tourism", "academic" từ 20-40 tuổi, vì:
    - Độ tuổi này đem lại nguồn doanh thu lớn cho khách sạn
    - Độ tuổi 40-61 mặc dù tỷ lệ hài lòng cao hơn, nhưng số lượng khách hàng không hài lòng vẫn còn nhiều nên sẽ khách sạn sẽ phần nào nhận được lợi nhuận gián tiếp dựa trên những giải pháp này

---

# 7. Nhìn lại quá trình làm đồ án

## Thành viên 1

- Họ tên: Lê Sỹ Thuần
- Đã gặp những khó khăn gì?   
Trong quá trình làm đồ án, việc tìm kiếm dữ liệu, bài toán dựa trên dữ liệu đó là công đoạn khó nhất. Tiếp theo là vấn đề đặt câu hỏi vì em cũng chưa có kinh nghiệm nhiều.  
Sử dụng Git để merge Jupyter notebook

- Có học được gì hữu ích?  
Hoàn thành được một đồ án Khoa học dữ liệu thực tế  mặc dù còn nhiều thiếu sót đã giúp em nắm rõ hơn về quy trình làm việc của nhà Khoa học dữ liệu.  
Phát triển, ôn tập lại một số kỹ năng xử lí dữ liệu đã học thông qua môn Lập trình Khoa học Dữ liệu, Khai thác dữ liệu và ứng dụng.  
Trau dồi thêm kinh nghiệm trong việc khai thác vấn đề từ dữ liệu, nhìn vấn đề cần khai thác nhanh hơn, rộng hơn

## Thành viên 2

- Họ tên: Nguyễn Minh Phúc
- Đã gặp những khó khăn gì?
    - Chọn tập dữ liệu phù hợp cho việc khám phá, phân tích
    - Chưa biết nhiều phương pháp, công cụ để phân tích dữ liệu hiệu quả
    - Thống nhất các quyết định của nhóm
- Có học được gì hữu ích?
    - Kỹ năng làm việc nhóm
    - Cách sử dụng Git để làm việc với nhóm
    - Công cụ hỗ trợ merge file jupyter notebook

## Nhóm
**Nếu có thêm thời gian thì sẽ làm gì?**
- Tìm hiểu thêm các phương pháp để phân tích dữ liệu
- Tìm hiểu thêm domain knowledge để có thể đặt câu hỏi có ích
- Phân tích dữ liệu kỹ hơn để khai thác dữ liệu tốt hơn.

# 8. Tài liệu tham khảo

- Các video bài giảng, slide của thầy Kiên
- Documents của các thư viện pandas, matplotlib, seaborn
- Stackoverflow
- 